In [2]:
import pyreadstat
import numpy as np
import pandas as pd 

## คำนวณ GINI แบบปกติ 

In [108]:
ses2011 , meta = pyreadstat.read_sav('./SES11/rec1_11.sav')
wt2011 , meta = pyreadstat.read_sav('./weight/weight11/weight11.sav')
sesforGINI2011 = pd.merge(ses2011, wt2011, on="hhid")

ses2013 , meta = pyreadstat.read_sav('./SES13/rec1_13.sav')
wt2013 , meta = pyreadstat.read_sav('./weight/weight13/weight13.sav')
sesforGINI2013 = pd.merge(ses2013, wt2013, on="hhid")

In [109]:
## Code GINI (packed) ปี 2011 และ 2013
## ต้องเชื่อม Weight POP ก่อน 

def gini (ses):
    ses.a16[ses['a16'] < 0] = 0
    ses = ses.sort_values(by=['a16'])
    ses['total_weight'] = sum(ses.wt_pop)
    ses['cumulative_weight'] = ses.wt_pop.cumsum()
    ses['cden'] = ses.cumulative_weight/ses.total_weight
    ses['Ybar'] = np.average(ses.a16, weights=ses.wt_pop)
    ses['mcden'] = np.average(ses.cden, weights=ses.wt_pop)
    gini = np.average(((ses.a16 - ses.Ybar)*(ses.cden - ses.mcden))/ses.Ybar, weights=ses.wt_pop)*2
    return gini
    
gini_2011 = gini(sesforGINI2011)
gini_2013 = gini(sesforGINI2013)

In [110]:
#2015
ses2015 , meta = pyreadstat.read_sav('./SES15/rec1_15.sav')
ses2015 = ses2015.rename(columns = {'NEW_HH_N':'hhid'})

wt2015 , meta = pyreadstat.read_sav('./weight/weight15/weight15.sav')
sesforGINI2015 = pd.merge(ses2015, wt2015, on="hhid")

#2017
ses2017 , meta = pyreadstat.read_sav('./SES17/rec1_17.sav')
ses2017 = ses2017.rename(columns = {'NEW_HH_NO':'hhid'})

wt2017 , meta = pyreadstat.read_sav('./weight/weight17/weight17.sav')
sesforGINI2017 = pd.merge(ses2017, wt2017, on="hhid")

#2019
ses2019 , meta = pyreadstat.read_sav('./SES19/REC1_19.sav')
ses2019 = ses2019.rename(columns = {'NEW_HH_NO':'hhid'})

wt2019 , meta = pyreadstat.read_sav('./weight/weight19/weight19.sav')
sesforGINI2019 = pd.merge(ses2019, wt2019, on="hhid")

#2020
ses2020 , meta = pyreadstat.read_sav('./SES20/ses2020q1234_record01 for TDRI.sav')
ses2020 = ses2020.rename(columns = {'NEW_HH_NO':'hhid'})

wt2020 , meta = pyreadstat.read_sav('./weight/weight20/wtpop2020.sav')
sesforGINI2020 = pd.merge(ses2020, wt2020, on="hhid")

In [111]:
## Code GINI (packed) ตั้งแต่ปี 2015
## ต้องเชื่อม Weight POP ก่อน 

def gini (ses):
    ses.A16[ses['A16'] < 0] = 0
    ses = ses.sort_values(by=['A16'])
    ses['total_weight'] = sum(ses.wt_pop)
    ses['cumulative_weight'] = ses.wt_pop.cumsum()
    ses['cden'] = ses.cumulative_weight/ses.total_weight
    ses['Ybar'] = np.average(ses.A16, weights=ses.wt_pop)
    ses['mcden'] = np.average(ses.cden, weights=ses.wt_pop)
    gini = np.average(((ses.A16 - ses.Ybar)*(ses.cden - ses.mcden))/ses.Ybar, weights=ses.wt_pop)*2
    return gini
    
gini_2015 = gini(sesforGINI2015)
gini_2017 = gini(sesforGINI2017)
gini_2019 = gini(sesforGINI2019)

In [112]:
## Code GINI (packed) ปี 2020
## ต้องเชื่อม Weight POP ก่อน 

def gini (ses):
    ses['b19'] = (ses.B18 - (ses.B15 + ses.B16 + ses.B10 + ses.B17))/ses.A04
    ses.b19[ses['b19'] < 0] = 0
    ses = ses.sort_values(by=['b19'])
    ses['total_weight'] = sum(ses.wt_pop)
    ses['cumulative_weight'] = ses.wt_pop.cumsum()
    ses['cden'] = ses.cumulative_weight/ses.total_weight
    ses['Ybar'] = np.average(ses.b19, weights=ses.wt_pop)
    ses['mcden'] = np.average(ses.cden, weights=ses.wt_pop)
    gini = np.average(((ses.b19 - ses.Ybar)*(ses.cden - ses.mcden))/ses.Ybar, weights=ses.wt_pop)*2
    return gini

gini_2020 = gini(sesforGINI2020)
gini_2020

0.41172259482130413

In [113]:
d = {'Year': [2011, 2013, 2015, 2017 , 2019 , 2020], 'GINI': [gini_2011, gini_2013, gini_2015, gini_2017, gini_2019, gini_2020]}
GINI = pd.DataFrame(data=d)
GINI

,Year,GINI
0,2011,0.484275
1,2013,0.465364
2,2015,0.445134
3,2017,0.452756
4,2019,0.430109
5,2020,0.411723


In [115]:
GINI.to_excel("GINI.xlsx")

In [116]:
## Decile 

from pandas._libs.lib import is_integer

def weighted_qcut(values, weights, q, **kwargs):
    'Return weighted quantile cuts from a given series, values.'
    if is_integer(q):
        quantiles = np.linspace(0, 1, q + 1)
    else:
        quantiles = q
    order = weights.iloc[values.argsort()].cumsum()
    bins = pd.cut(order / order.iloc[-1], quantiles, **kwargs)
    return bins.sort_index()

sesforGINI2011['Decile_by_income'] = weighted_qcut(sesforGINI2011['a16'], sesforGINI2011['wt_pop'], 10, labels=False)
sesforGINI2013['Decile_by_income'] = weighted_qcut(sesforGINI2013['a16'], sesforGINI2013['wt_pop'], 10, labels=False)
sesforGINI2015['Decile_by_income'] = weighted_qcut(sesforGINI2015['A16'], sesforGINI2015['wt_pop'], 10, labels=False)
sesforGINI2017['Decile_by_income'] = weighted_qcut(sesforGINI2017['A16'], sesforGINI2017['wt_pop'], 10, labels=False)
sesforGINI2019['Decile_by_income'] = weighted_qcut(sesforGINI2019['A16'], sesforGINI2019['wt_pop'], 10, labels=False)
sesforGINI2020['Decile_by_income'] = weighted_qcut(sesforGINI2020['b19'], sesforGINI2020['wt_pop'], 10, labels=False)

In [117]:
Income_Decile_2011 = pd.DataFrame(((sesforGINI2011.wt_pop*sesforGINI2011.a16).groupby(sesforGINI2011['Decile_by_income']).sum())/(sesforGINI2011.wt_pop.groupby(sesforGINI2011['Decile_by_income']).sum()))
Income_Decile_2013 = pd.DataFrame(((sesforGINI2013.wt_pop*sesforGINI2013.a16).groupby(sesforGINI2013['Decile_by_income']).sum())/(sesforGINI2013.wt_pop.groupby(sesforGINI2013['Decile_by_income']).sum()))
Income_Decile_2015 = pd.DataFrame(((sesforGINI2015.wt_pop*sesforGINI2015.A16).groupby(sesforGINI2015['Decile_by_income']).sum())/(sesforGINI2015.wt_pop.groupby(sesforGINI2015['Decile_by_income']).sum()))
Income_Decile_2017 = pd.DataFrame(((sesforGINI2017.wt_pop*sesforGINI2017.A16).groupby(sesforGINI2017['Decile_by_income']).sum())/(sesforGINI2017.wt_pop.groupby(sesforGINI2017['Decile_by_income']).sum()))
Income_Decile_2019 = pd.DataFrame(((sesforGINI2019.wt_pop*sesforGINI2019.A16).groupby(sesforGINI2019['Decile_by_income']).sum())/(sesforGINI2019.wt_pop.groupby(sesforGINI2019['Decile_by_income']).sum()))
Income_Decile_2020 = pd.DataFrame(((sesforGINI2020.wt_pop*sesforGINI2020.b19).groupby(sesforGINI2020['Decile_by_income']).sum())/(sesforGINI2020.wt_pop.groupby(sesforGINI2020['Decile_by_income']).sum()))

In [118]:
Income_Decile_2011_sum = pd.DataFrame(((sesforGINI2011.wt_pop*sesforGINI2011.a16).groupby(sesforGINI2011['Decile_by_income']).sum()))
Income_Decile_2013_sum = pd.DataFrame(((sesforGINI2013.wt_pop*sesforGINI2013.a16).groupby(sesforGINI2013['Decile_by_income']).sum()))
Income_Decile_2015_sum = pd.DataFrame(((sesforGINI2015.wt_pop*sesforGINI2015.A16).groupby(sesforGINI2015['Decile_by_income']).sum()))
Income_Decile_2017_sum = pd.DataFrame(((sesforGINI2017.wt_pop*sesforGINI2017.A16).groupby(sesforGINI2017['Decile_by_income']).sum()))
Income_Decile_2019_sum = pd.DataFrame(((sesforGINI2019.wt_pop*sesforGINI2019.A16).groupby(sesforGINI2019['Decile_by_income']).sum()))
Income_Decile_2020_sum = pd.DataFrame(((sesforGINI2020.wt_pop*sesforGINI2020.b19).groupby(sesforGINI2020['Decile_by_income']).sum()))

In [119]:
import xlsxwriter
writer = pd.ExcelWriter('Income_Decile.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
Income_Decile_2011.to_excel(writer, sheet_name='2011')
Income_Decile_2013.to_excel(writer, sheet_name='2013')
Income_Decile_2015.to_excel(writer, sheet_name='2015')
Income_Decile_2017.to_excel(writer, sheet_name='2017')
Income_Decile_2019.to_excel(writer, sheet_name='2019')
Income_Decile_2020.to_excel(writer, sheet_name='2020')
# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [120]:
writer = pd.ExcelWriter('Income_Decile_sum.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
Income_Decile_2011_sum.to_excel(writer, sheet_name='2011')
Income_Decile_2013_sum.to_excel(writer, sheet_name='2013')
Income_Decile_2015_sum.to_excel(writer, sheet_name='2015')
Income_Decile_2017_sum.to_excel(writer, sheet_name='2017')
Income_Decile_2019_sum.to_excel(writer, sheet_name='2019')
Income_Decile_2020_sum.to_excel(writer, sheet_name='2020')
# Close the Pandas Excel writer and output the Excel file.
writer.save()

## Gini แบบตัดเงินโอนภาครัฐ

In [121]:
ses2011 , meta = pyreadstat.read_sav('./SES11/rec1_11.sav')
wt2011 , meta = pyreadstat.read_sav('./weight/weight11/weight11.sav')
sesforGINI2011 = pd.merge(ses2011, wt2011, on="hhid")

ses2013 , meta = pyreadstat.read_sav('./SES13/rec1_13.sav')
wt2013 , meta = pyreadstat.read_sav('./weight/weight13/weight13.sav')
sesforGINI2013 = pd.merge(ses2013, wt2013, on="hhid")

#2015
ses2015 , meta = pyreadstat.read_sav('./SES15/rec1_15.sav')
ses2015 = ses2015.rename(columns = {'NEW_HH_N':'hhid'})

wt2015 , meta = pyreadstat.read_sav('./weight/weight15/weight15.sav')
sesforGINI2015 = pd.merge(ses2015, wt2015, on="hhid")

#2017
ses2017 , meta = pyreadstat.read_sav('./SES17/rec1_17.sav')
ses2017 = ses2017.rename(columns = {'NEW_HH_NO':'hhid'})

wt2017 , meta = pyreadstat.read_sav('./weight/weight17/weight17.sav')
sesforGINI2017 = pd.merge(ses2017, wt2017, on="hhid")

#2019
ses2019 , meta = pyreadstat.read_sav('./SES19/REC1_19.sav')
ses2019 = ses2019.rename(columns = {'NEW_HH_NO':'hhid'})

wt2019 , meta = pyreadstat.read_sav('./weight/weight19/weight19.sav')
sesforGINI2019 = pd.merge(ses2019, wt2019, on="hhid")

#2020
ses2020 , meta = pyreadstat.read_sav('./SES20/ses2020q1234_record01 for TDRI.sav')
ses2020 = ses2020.rename(columns = {'NEW_HH_NO':'hhid'})

wt2020 , meta = pyreadstat.read_sav('./weight/weight20/wtpop2020.sav')
sesforGINI2020 = pd.merge(ses2020, wt2020, on="hhid")

In [122]:
## Code GINI (packed) ปี 2011 และ 2013 ตัดเงินโอนภาครัฐ
## ต้องเชื่อม Weight POP ก่อน 

def gini (ses):
    ses['a16'] = ses['a16'] - (ses['a30']/ses['a04'])
    ses.a16[ses['a16'] < 0] = 0
    ses = ses.sort_values(by=['a16'])
    ses['total_weight'] = sum(ses.wt_pop)
    ses['cumulative_weight'] = ses.wt_pop.cumsum()
    ses['cden'] = ses.cumulative_weight/ses.total_weight
    ses['Ybar'] = np.average(ses.a16, weights=ses.wt_pop)
    ses['mcden'] = np.average(ses.cden, weights=ses.wt_pop)
    gini = np.average(((ses.a16 - ses.Ybar)*(ses.cden - ses.mcden))/ses.Ybar, weights=ses.wt_pop)*2
    return gini
    
gini_2011_nontransfer = gini(sesforGINI2011)
gini_2013_nontransfer = gini(sesforGINI2013)

In [123]:
## Code GINI (packed) ตั้งแต่ปี 2015 ตัดเงินโอนภาครัฐ
## ต้องเชื่อม Weight POP ก่อน 

def gini (ses):
    ses['A16'] = ses['A16'] - (ses['A30']/ses['A04'])
    ses.A16[ses['A16'] < 0] = 0
    ses = ses.sort_values(by=['A16'])
    ses['total_weight'] = sum(ses.wt_pop)
    ses['cumulative_weight'] = ses.wt_pop.cumsum()
    ses['cden'] = ses.cumulative_weight/ses.total_weight
    ses['Ybar'] = np.average(ses.A16, weights=ses.wt_pop)
    ses['mcden'] = np.average(ses.cden, weights=ses.wt_pop)
    gini = np.average(((ses.A16 - ses.Ybar)*(ses.cden - ses.mcden))/ses.Ybar, weights=ses.wt_pop)*2
    return gini

gini_2015_nontransfer = gini(sesforGINI2015)
gini_2017_nontransfer = gini(sesforGINI2017)
gini_2019_nontransfer = gini(sesforGINI2019)

In [124]:
## Code GINI (packed) ปี 2020 ตัดเงินโอนภาครัฐ
## ต้องเชื่อม Weight POP ก่อน 

def gini (ses):
    ses['b19'] = (ses.B18 - (ses.B15 + ses.B16 + ses.B10 + ses.B17+ ses.B08))/ses.A04
    ses.b19[ses['b19'] < 0] = 0
    ses = ses.sort_values(by=['b19'])
    ses['total_weight'] = sum(ses.wt_pop)
    ses['cumulative_weight'] = ses.wt_pop.cumsum()
    ses['cden'] = ses.cumulative_weight/ses.total_weight
    ses['Ybar'] = np.average(ses.b19, weights=ses.wt_pop)
    ses['mcden'] = np.average(ses.cden, weights=ses.wt_pop)
    gini = np.average(((ses.b19 - ses.Ybar)*(ses.cden - ses.mcden))/ses.Ybar, weights=ses.wt_pop)*2
    return gini

gini_2020_nontransfer = gini(sesforGINI2020)

In [125]:
d_nontransfer = {'Year': [2011, 2013, 2015, 2017 , 2019, 2020], 
                 'GINI': [gini_2011_nontransfer, gini_2013_nontransfer, gini_2015_nontransfer, gini_2017_nontransfer, gini_2019_nontransfer, gini_2020_nontransfer]}
GINI_nontransfer = pd.DataFrame(data=d_nontransfer)
GINI_nontransfer

,Year,GINI
0,2011,0.493275
1,2013,0.472980
2,2015,0.454666
3,2017,0.464563
4,2019,0.445235
5,2020,0.444788


In [126]:
GINI_nontransfer.to_excel("GINI_nontransfer.xlsx")

In [127]:
sesforGINI2011['Decile_by_income'] = weighted_qcut(sesforGINI2011['a16'], sesforGINI2011['wt_pop'], 10, labels=False)
sesforGINI2013['Decile_by_income'] = weighted_qcut(sesforGINI2013['a16'], sesforGINI2013['wt_pop'], 10, labels=False)
sesforGINI2015['Decile_by_income'] = weighted_qcut(sesforGINI2015['A16'], sesforGINI2015['wt_pop'], 10, labels=False)
sesforGINI2017['Decile_by_income'] = weighted_qcut(sesforGINI2017['A16'], sesforGINI2017['wt_pop'], 10, labels=False)
sesforGINI2019['Decile_by_income'] = weighted_qcut(sesforGINI2019['A16'], sesforGINI2019['wt_pop'], 10, labels=False)
sesforGINI2020['Decile_by_income'] = weighted_qcut(sesforGINI2020['b19'], sesforGINI2020['wt_pop'], 10, labels=False)

In [128]:
Income_Decile_2011 = pd.DataFrame(((sesforGINI2011.wt_pop*sesforGINI2011.a16).groupby(sesforGINI2011['Decile_by_income']).sum())/(sesforGINI2011.wt_pop.groupby(sesforGINI2011['Decile_by_income']).sum()))
Income_Decile_2013 = pd.DataFrame(((sesforGINI2013.wt_pop*sesforGINI2013.a16).groupby(sesforGINI2013['Decile_by_income']).sum())/(sesforGINI2013.wt_pop.groupby(sesforGINI2013['Decile_by_income']).sum()))
Income_Decile_2015 = pd.DataFrame(((sesforGINI2015.wt_pop*sesforGINI2015.A16).groupby(sesforGINI2015['Decile_by_income']).sum())/(sesforGINI2015.wt_pop.groupby(sesforGINI2015['Decile_by_income']).sum()))
Income_Decile_2017 = pd.DataFrame(((sesforGINI2017.wt_pop*sesforGINI2017.A16).groupby(sesforGINI2017['Decile_by_income']).sum())/(sesforGINI2017.wt_pop.groupby(sesforGINI2017['Decile_by_income']).sum()))
Income_Decile_2019 = pd.DataFrame(((sesforGINI2019.wt_pop*sesforGINI2019.A16).groupby(sesforGINI2019['Decile_by_income']).sum())/(sesforGINI2019.wt_pop.groupby(sesforGINI2019['Decile_by_income']).sum()))
Income_Decile_2020 = pd.DataFrame(((sesforGINI2020.wt_pop*sesforGINI2020.b19).groupby(sesforGINI2020['Decile_by_income']).sum())/(sesforGINI2020.wt_pop.groupby(sesforGINI2020['Decile_by_income']).sum()))

In [129]:
Income_Decile_2011_sum = pd.DataFrame(((sesforGINI2011.wt_pop*sesforGINI2011.a16).groupby(sesforGINI2011['Decile_by_income']).sum()))
Income_Decile_2013_sum = pd.DataFrame(((sesforGINI2013.wt_pop*sesforGINI2013.a16).groupby(sesforGINI2013['Decile_by_income']).sum()))
Income_Decile_2015_sum = pd.DataFrame(((sesforGINI2015.wt_pop*sesforGINI2015.A16).groupby(sesforGINI2015['Decile_by_income']).sum()))
Income_Decile_2017_sum = pd.DataFrame(((sesforGINI2017.wt_pop*sesforGINI2017.A16).groupby(sesforGINI2017['Decile_by_income']).sum()))
Income_Decile_2019_sum = pd.DataFrame(((sesforGINI2019.wt_pop*sesforGINI2019.A16).groupby(sesforGINI2019['Decile_by_income']).sum()))
Income_Decile_2020_sum = pd.DataFrame(((sesforGINI2020.wt_pop*sesforGINI2020.b19).groupby(sesforGINI2020['Decile_by_income']).sum()))

In [130]:
writer = pd.ExcelWriter('Income_Decile_nontransfer.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
Income_Decile_2011.to_excel(writer, sheet_name='2011')
Income_Decile_2013.to_excel(writer, sheet_name='2013')
Income_Decile_2015.to_excel(writer, sheet_name='2015')
Income_Decile_2017.to_excel(writer, sheet_name='2017')
Income_Decile_2019.to_excel(writer, sheet_name='2019')
Income_Decile_2020.to_excel(writer, sheet_name='2020')
# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [131]:
writer = pd.ExcelWriter('Income_Decile_nontransfer_sum.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
Income_Decile_2011_sum.to_excel(writer, sheet_name='2011')
Income_Decile_2013_sum.to_excel(writer, sheet_name='2013')
Income_Decile_2015_sum.to_excel(writer, sheet_name='2015')
Income_Decile_2017_sum.to_excel(writer, sheet_name='2017')
Income_Decile_2019_sum.to_excel(writer, sheet_name='2019')
Income_Decile_2020_sum.to_excel(writer, sheet_name='2020')
# Close the Pandas Excel writer and output the Excel file.
writer.save()